# Command Based Programming

The previous

## Lambdas

### Functional Interfaces and Anonymous Functions

### Method References

## Why Does it Matter: The Issue with Just Having Subsystems

Having *just* `Subsystem` as an abstraction still falls short in a big way:

## Commands 

### A Basic Structure / The Manual Way 

#### A Command's Constituent Components

#### CommandBase and A Worked Example

### A Better, Functional Way

#### Command Factories


#### Some Selected Examples

## Events and Triggers : An Addendum to Inputs 

### Limits of Polling Boolean Inputs

### Eventloop and BooleanEven Objects

### Trigger Objects

***We still have to manually check each input case and map that to some chain of actions!!***

```java
@Override
public void robotPeriodic() {
    if (/* input case 1*/){
        // do something w/ some subsystems
    } else if (/* input case 2 */) {
        // do something else w/ other subsystems
    } ...
}
```

We see that with enough cases and complexity, this pattern becomes unsustainable

(note)We also have to manually deal with conflicts in our design; for example if we accidently had 2 input cases that needed to use the same subsystem at the same time.... things would break

## Commands

> Commands are simple state machines that perform high-level robot functions using the methods defined by subsystems. Commands can be either idle, in which they do nothing, or scheduled, in which the scheduler will execute a specific set of the command’s code depending on the state of the command. The CommandScheduler recognizes scheduled commands as being in one of three states: initializing, executing, or ending. Commands specify what is done in each of these states through the initialize(), execute() and end() methods. 
>
> \- WPIlib documentation

Commands are an abstraction over actions/ responsibilities that can be performed at any given time. 

### Creating a Command
To create a command, extend the `CommandBase` class, and override a few important methods, those being `initialize`, `execute`, `end`, and `isFinished`
```java
import edu.wpi.first.wpilibj2.command.CommandBase;

class ExampleCommand extends CommandBase {
  @Override
  public void initialize() {}

  @Override
  public void execute() {}

  @Override
  public void end(boolean interrupted) {}

  @Override
  public boolean isFinished(){ return false; }
}
```

`initialize` : when the command starts executing (similar to the `<x>Init` methods)
  - set up everything needed for the execution of the command

`execute` : while the command is running (similar to the `<x>Periodic` methods)
  - the main thing  the action/command is trying to accomplish

`end` : directly after the command stops running
  - clean up and finalize the command (bring it to a conclusion)
  - the `interrupted` parameter is whether it finished successfully (`false`) or was shut down by the scheduler (`true`)

`isFinished` checks whether the command has accomplished its goal
  - if it returns `true`, then `end` will be called with `false` and execution for the command will stop
  - always returning false makes it never stop unless interrupted

#### Note : Default Behaviors

by default the `end`, `initialize`, and `execute` methods do nothing, and `isFinished` always returns false, so a very simple command that always runs in the background can get away with just overriding `execute`. 

We'll expand on this later by introducing default commands

### Binding Subsystems

A Command like that on its own isn't very helpful, commands need to interact with subsystems to perform some useful action. To specify which subsystems, we need to call the `addRequirements` method of `CommandBase` on all the subsystems that we need (usually in the constructor).

```java
class ExampleCommand extends CommandBase {
    public ExampleCommand(){
        this.addRequirements(subsys1, subsys2, ...);
    }
    @Override
    public void execute() {
        // do stuff
    }
}
```

But how do we get those subsystems??? 

Note : If we created these subsystem in the command, then we'd have duplicate sub-systems for the same components!

We can do it cleanly by passing subsystem objects to the constructor:

```java
class ExampleCommand extends CommandBase {
    public ExampleCommand(SubSystemName subsys1, ...){
        this.addRequirements(subsys1, ...);
    }
}
```

This is an example of a pattern called `dependency injection` where things that the command needs (dependencies) are provided (injected). This helps seperate the creation and usage of out subsystems, in a nod to the principle of `seperation of concerns` from before.

### Default Commands

One particular command usage for a command is to have it always run. These commands often handle core background tasks that are essential to the function of the robot, like a `drive` command that controls robot motion.

for these types of commands, we can make that command a `default` command of the subsystem:

```java
class ExampleCommand extends CommandBase {
    public ExampleCommand(SubSystemName subsys1){
        this.addRequirements(subsys1);
        // this refers to the current object
        subsys1.setDefaultCommand(this);
    }
}
```

If the subsystem is not the requirement of some other command, then this default command will be executed.

### Example Command

```java
public class ClimberCommand extends CommandBase {
    private final Climber climber;
    private final UserAnalog speed;
    // climber is the subsystem
    public ClimberCommand(Climber climber, UserAnalog speed) {
        this.climber = climber;
        this.speed = speed;
        // works b/c addRequirements is a method of CommandBase
        this.addRequirements(climber);
        // making it a default command
        climber.setDefaultCommand(this);
    }

    @Override
    public void execute() {
        climber.extend(speed.get(), false);
    }
}
```

## The Command Scheduler

Now that we have our command, how do we run it? We defined methods for what to do in different cases with `initalize`, `execute`, and `end`, but we never called them anywhere.

Luckily, there is something that handles this, as well as the general logic of commands for us: the Command Scheduler.

Since there should only be 1 instance of the command scheduler, we get it through an `getInstance` static method, e.g.
```java
// CommandScheduler is what is known as a "singleton"
CommandScheduler.getInstance().run();
```
the CommandScheduler instance has 2 methods of use to us: `schedule` and `run`

`schedule` sets up a command to be run, and checks whether it makes sense to run the command, and then takes appropriate actions. These checks include:
 - whether the command is already scheduled
 - whether the subsystems required aren't already being used
   - if they are being used, whether the other commands be interrupted
  

`run` runs through 1 cycle of the command scheduler, which involves:
 - running each Subsystem `periodic` method
 - executing all scheduled commands
 - seeing which commands have finished and remove them from the schedule

### Using commands

to use our commands, we'd only need to call `schedule` on them when the correct condition is met. For our default commands, this isn't necessary. For example, if we wanted some command to run at the very start, then we could do: 

```java
@Override
public void robotInit() {
    CommandScheduler.getInstance().schedule(
        new SomeCommand(new SomeSubsystem(...))
    );
}
```

We'd also need to reliably run our `CommandScheduler` each cycle, so we would call the `run` method from `robotPeriodic`:

```java
@Override
public void robotPeriodic() {
    CommandScheduler.getInstance().run();
}
```

## Digression : Simplicity & Abstraction 

Both by using the command scheduler as opposed to manually managing our subsystems, and by using subsystems instead of working with individual electronics, we saw that our code got a lot cleaner, and we were also able to better understand its logic. 

(e.g. It's much easier to understand `drive the robot` as a high level instruction than it is to understand what happens when individual motors are spun)

This idea of code being much more understandable as more and more gets hidden and abstracted away from us is a common one in computing in general. In very many applications, it's often important that we see which parts of the problem are important for us as people to reason with, and which parts can be handled without us needing to think about it explicitly.

## Grouping Commands

We've seen how to work with individual commands, but there are many cases where we want two commands to run at the same time (in parallel), or one after the other (in sequence), or in any number of ways depending on the circumstances.

Using the existing features, we can use the `initalize` and `end` methods to schedule parallel and sequential commands respectively, but WPILib offers us a much more elegant solution...

### Sequencing commands

To run one command after another, we can do one of two things.

1. we can a `Command`'s `andThen`/`beforeStarting` methods to run commands after/before some command. (Note that `andThen` can take > 1 commands, but `beforeStarting` only takes 1)

```java
// a chain of commands
command1
  .andThen(command2, command3)
  .beforeStarting(command0);
```


2. we can create a `SequentialCommandGroup` with all the commands we want to run:

```java
new SequentialCommandGroup(
  command0,
  command1,
  command2,
  command3
);
```

the `andThen` and `beforeStarting` methods also return a `SequentialCommandGroup` object, so both these methods can be equally good. `SequentialCommandGroup`s are themselves commands, so they also be used wherever any individual `Command` is needed. (they extend a class which extends CommandBase) 

### Parallelizing Commands

Unlike sequential commands, which just involve running one command, and then running another when the first command finishes, there are many ways to parallelize commands, since those individual commands can end at different times.

#### We Could...

- end when one of the commands ends
- end when all of the commands end
- end when a specific command ends

#### Luckily, we can do all 3

1. to end when the one command finishes, we can use the `raceWith` method
2. to end when the all commands finish, we can use the `alongWith` method
3. to end when a specific command finishes, we can use the `deadlineWith​` method

#### Direct Construction

Similar to sequenced command groups, we can also directly construct parallel command group object instead of using those methods. As there are three ways to parallelize commands, there are 3 different parallel command groups that we can use.

1. ParallelRaceGroup
2. ParallelCommandGroup
3. ParallelDeadlineGroup

The methods from before return values of these types, and values of these parallel groups, like values of sequential command groups, are themselves valid commands

### The in-depth description

https://docs.wpilib.org/en/stable/docs/software/commandbased/structuring-command-based-project.html

### Documentation

- https://first.wpi.edu/wpilib/allwpilib/docs/release/java/index.html
- https://api.ctr-electronics.com/phoenix/release/java/index.html
- https://codedocs.revrobotics.com/java/com/revrobotics/package-summary.html